In [1]:
from PIL import Image, ImageDraw, ImageFont
from matplotlib import pyplot as plt
import numpy as np

In [2]:
def map_value(x, in_min, in_max, out_min, out_max):
        return ((x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min)  

In [3]:
class VisualSimplex:
    def __init__(self, node1, node2, node3, color_x, color_y, color_stress):
        self.x1 = node1.mapped_x
        self.y1 = node1.mapped_y
        self.x2 = node2.mapped_x
        self.y2 = node2.mapped_y
        self.x3 = node3.mapped_x
        self.y3 = node3.mapped_y
        
        self.color_x = color_x
        self.color_y = color_y
        
        self.color_stress = color_stress
    
    def draw_simplex_x(self, drawer):
        coords_array = [(self.x1, self.y1), (self.x2, self.y2), (self.x3, self.y3)]
        drawer.polygon(coords_array, fill = self.color_x, outline = (0, 0, 0))
        
    def draw_simplex_y(self, drawer):
        coords_array = [(self.x1, self.y1), (self.x2, self.y2), (self.x3, self.y3)]
        drawer.polygon(coords_array, fill = self.color_y, outline = (0, 0, 0))
        
    def draw_simplex_stress(self, drawer):
        coords_array = [(self.x1, self.y1), (self.x2, self.y2), (self.x3, self.y3)]
        outline_R = max(0, self.color_stress[0] - 40)
        outline_G = max(0, self.color_stress[1] - 40)
        outline_B = max(0, self.color_stress[2] - 40)
        out_col = (outline_R, outline_G, outline_B)
        drawer.polygon(coords_array, fill = self.color_stress, outline = out_col)

In [4]:
class Node:
    def __init__(self, nx, ny, value_x, value_y):
        self.unmapped_x = nx
        self.unmapped_y = ny
        self.value_x = value_x
        self.value_y = value_y
        
    def map_coords(self, real_x_min, real_x_max, real_y_min, real_y_max, draw_x_min, draw_x_max, draw_y_min, draw_y_max):
        self.mapped_x = int(map_value(self.unmapped_x, real_x_min, real_x_max, draw_x_min, draw_x_max))
        self.mapped_y = int(map_value(self.unmapped_y, real_y_min, real_y_max, draw_y_min, draw_y_max))

In [5]:
class PostProcessor:
    def __init__(self, bg_color, low_color, high_color):
        self.canv = Image.new('RGB', (900, 900), bg_color)
        self.draw = ImageDraw.Draw(self.canv)
        
        self.simplexes = {}
        
        self.nodes = {}
        self.elem_nodes = {}
        
        self.min_x = 1e9
        self.min_y = 1e9
        self.max_x = -1e9
        self.max_y = -1e9
        
        self.min_value_x = 1e9
        self.min_value_y = 1e9
        self.max_value_x = -1e9
        self.max_value_y = -1e9
        
        self.min_stress = 1e9
        self.max_stress = -1e9
        
        # Pixels margin on canvas
        self.margin_x = None
        self.margin_y = None
        
        # Color gradient
        self.low_color = low_color
        self.high_color = high_color
        
        self.elem_stresses = {}
        
    def update_borders(self, nx, ny, val_x, val_y):
        if (nx > self.max_x):
            self.max_x = nx
            
        if (nx < self.min_x):
            self.min_x = nx
            
        if (ny > self.max_y):
            self.max_y = ny
            
        if (ny < self.min_y):
            self.min_y = ny
            
            
        if (val_x > self.max_value_x):
            self.max_value_x = val_x
            
        if (val_x < self.min_value_x):
            self.min_value_x = val_x
            
        if (val_y > self.max_value_y):
            self.max_value_y = val_y
            
        if (val_y < self.min_value_y):
            self.min_value_y = val_y
        
    def parse_file(self, file_path):
        input_file = open(file_path, 'r')
    
        # Reading nodes data
        n_nodes = int(input_file.readline())
        for i in range(n_nodes):
            data = input_file.readline().split(' ')
            node_id = int(data[0])
            u_x = float(data[1])
            u_y = float(data[2])
            node_x = float(data[3])
            node_y = float(data[4])
            
            new_node = Node(node_x, node_y, u_x, u_y)
            self.nodes[node_id] = new_node
            
            self.update_borders(node_x, node_y, u_x, u_y)
            
        # Reading elems data
        n_elems = int(input_file.readline())
        for i in range(n_elems):
            data = input_file.readline().split(' ')
            elem_id = int(data[0])
            node_1, node_2, node_3 = tuple(map(int, data[1:4]))
            elem_stress = abs(float(data[4]))                            # Absolute value here!
            elem_strain = float(data[5])
            
            if (elem_stress > self.max_stress):
                self.max_stress = elem_stress
            if (elem_stress < self.min_stress):
                self.min_stress = elem_stress
            
            self.elem_nodes[elem_id] = [node_1, node_2, node_3]
            self.elem_stresses[elem_id] = elem_stress
            
        input_file.close()
        
    def calc_margins(self):
        x_range = self.max_x - self.min_x
        y_range = self.max_y - self.min_y
        
        if (x_range >= y_range):
            self.margin_x = 10 # Pix
            self.margin_y = 10 + int((890 * (1 - y_range/x_range)) / 2)
        else:
            self.margin_y = 10 # Pix
            self.margin_x = 10 + int((890 * (1 - x_range/y_range)) / 2)
            
    def map_colors(self, value_x, value_y):
        min_R, min_G, min_B = self.low_color
        max_R, max_G, max_B = self.high_color
        
        color_x_R = int(map_value(value_x, self.min_value_x, self.max_value_x, min_R, max_R))
        color_x_G = int(map_value(value_x, self.min_value_x, self.max_value_x, min_G, max_G))
        color_x_B = int(map_value(value_x, self.min_value_x, self.max_value_x, min_B, max_B))
        
        color_y_R = int(map_value(value_y, self.min_value_y, self.max_value_y, min_R, max_R))
        color_y_G = int(map_value(value_y, self.min_value_y, self.max_value_y, min_G, max_G))
        color_y_B = int(map_value(value_y, self.min_value_y, self.max_value_y, min_B, max_B))
        
        color_x = (color_x_R, color_x_G, color_x_B)
        color_y = (color_y_R, color_y_G, color_y_B)
        
        return color_x, color_y
        
        
    def form_simplexes(self):
        
        value_arr = np.linspace(0, 1, 100)
        colors = plt.cm.viridis(value_arr)
        
        low_color = colors[0]
        low_color_R = int(255*low_color[0])
        low_color_G = int(255*low_color[1])
        low_color_B = int(255*low_color[2])
        
        high_color = colors[-1]
        high_color_R = int(255*high_color[0])
        high_color_G = int(255*high_color[1])
        high_color_B = int(255*high_color[2])
        
        self.low_color = (low_color_R, low_color_G, low_color_B)
        self.high_color = (high_color_R, high_color_G, high_color_B)
        
        for elem_key in list(self.elem_nodes.keys()):
            node_id_1, node_id_2, node_id_3 = tuple(self.elem_nodes[elem_key])
            
            self.nodes[node_id_1].map_coords(self.min_x, self.max_x, self.min_y, self.max_y,
                                                     self.margin_x, 900-self.margin_x, self.margin_y, 900-self.margin_y)
            self.nodes[node_id_2].map_coords(self.min_x, self.max_x, self.min_y, self.max_y,
                                                     self.margin_x, 900-self.margin_x, self.margin_y, 900-self.margin_y)
            self.nodes[node_id_3].map_coords(self.min_x, self.max_x, self.min_y, self.max_y,
                                                     self.margin_x, 900-self.margin_x, self.margin_y, 900-self.margin_y)
            
            avg_value_x = (self.nodes[node_id_1].value_x + self.nodes[node_id_2].value_x + self.nodes[node_id_3].value_x) / 3
            avg_value_y = (self.nodes[node_id_1].value_y + self.nodes[node_id_2].value_y + self.nodes[node_id_3].value_y) / 3
            
            color_x, color_y = self.map_colors(avg_value_x, avg_value_y)
            
            e_stress = self.elem_stresses[elem_key]
            
            stress_mapped = map_value(e_stress, self.min_stress, self.max_stress, 0, 1)
            closest_color_i = np.argmin(np.abs(value_arr - stress_mapped))
            color = colors[closest_color_i]
            
            color_R = int(255*color[0])
            color_G = int(255*color[1])
            color_B = int(255*color[2])
            
            #print(color_R, color_G, color_B)
            
            new_simplex = VisualSimplex(self.nodes[node_id_1], self.nodes[node_id_2], self.nodes[node_id_3],
                                       color_x, color_y, (color_R, color_G, color_B))
            
            self.simplexes[elem_key] = new_simplex
            
    def invert_simplexes(self):
        for simplex_ID in list(self.simplexes.keys()):
            self.simplexes[simplex_ID].y1 = 900 - self.simplexes[simplex_ID].y1
            self.simplexes[simplex_ID].y2 = 900 - self.simplexes[simplex_ID].y2
            self.simplexes[simplex_ID].y3 = 900 - self.simplexes[simplex_ID].y3
            
    def draw_mesh_y(self):
        for simplex_ID in list(self.simplexes.keys()):
            self.simplexes[simplex_ID].draw_simplex_y(self.draw)
            
    def draw_mesh_y(self):
        for simplex_ID in list(self.simplexes.keys()):
            self.simplexes[simplex_ID].draw_simplex_x(self.draw)
            
    def draw_mesh_stress(self):
        for simplex_ID in list(self.simplexes.keys()):
            self.simplexes[simplex_ID].draw_simplex_stress(self.draw)
            
    def draw_legend_y(self):
        legend_size = 20
        # Low color:
        self.draw.rectangle([(15, 15), (15 + legend_size, 15 + legend_size)], fill = self.low_color, outline = 'blue')
        # High color:
        self.draw.rectangle([(15, 55), (15 + legend_size, 55 + legend_size)], fill = self.high_color, outline = 'blue')
        
        myFont = ImageFont.truetype('font.otf', 13)
        
        self.draw.text((55, 15), f'low = {self.min_value_y:.4e}', (0, 0, 0), font = myFont)
        self.draw.text((55, 55), f'high = {self.max_value_y:.4e}', (0, 0, 0), font = myFont)
        
    def draw_legend_x(self):
        legend_size = 20
        # Low color:
        self.draw.rectangle([(15, 15), (15 + legend_size, 15 + legend_size)], fill = self.low_color, outline = 'blue')
        # High color:
        self.draw.rectangle([(15, 55), (15 + legend_size, 55 + legend_size)], fill = self.high_color, outline = 'blue')
        
        myFont = ImageFont.truetype('font.otf', 13)
        
        self.draw.text((55, 15), f'low = {self.min_value_x:.4e}', (0, 0, 0), font = myFont)
        self.draw.text((55, 55), f'high = {self.max_value_x:.4e}', (0, 0, 0), font = myFont)
        
    def draw_legend_stress(self):
        legend_size = 20
        # Low color:
        self.draw.rectangle([(15, 15), (15 + legend_size, 15 + legend_size)], fill = self.low_color, outline = 'blue')
        # High color:
        self.draw.rectangle([(15, 55), (15 + legend_size, 55 + legend_size)], fill = self.high_color, outline = 'blue')
        
        myFont = ImageFont.truetype('font.otf', 13)
        
        self.draw.text((55, 15), f'low = {self.min_stress:.4e}', (0, 0, 0), font = myFont)
        self.draw.text((55, 55), f'high = {self.max_stress:.4e}', (0, 0, 0), font = myFont)
        
    def draw_grid(self, grid_step):
        # Grid along oY:
        y_start = 50
        while (y_start < 900):
            self.dotted_hori(y_start)
            self.dotted_vert(y_start)
            y_start += grid_step
            
            
    def draw_centrer(self):
        self.draw.line([(450, 0), (450, 900)], fill = (100, 100, 100))
        self.draw.line([(0, 450), (900, 450)], fill = (100, 100, 100))
        
    def dotted_hori(self, y):
        x_start = 0
        
        while (x_start < 900):
            self.draw.line([(x_start, y), (x_start + 1, y)], fill = (0,0,0))
            x_start += 7
        
    def dotted_vert(self, x):
        y_start = 0
        
        while (y_start < 900):
            self.draw.line([(x, y_start), (x, y_start + 1)], fill = (0,0,0))
            y_start += 7
            
    def save(self):
        self.canv.save('Result.jpg')

In [6]:
low_color = (10, 0, 0)
high_color = (255, 255, 180)

pp = PostProcessor((255, 255, 255), low_color, high_color)

In [7]:
pp.parse_file('Kirsh_1_Displ.txt')

In [8]:
pp.calc_margins()

In [9]:
pp.form_simplexes()
pp.invert_simplexes()

In [10]:
pp.draw_grid(100)
pp.draw_mesh_stress()
pp.draw_legend_stress()

In [11]:
pp.canv.show()
pp.save()